In [148]:
import matplotlib
matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
import scipy as sp
import numpy as np
from scipy.sparse import linalg as ln
from scipy import sparse as sparse
import matplotlib.animation as animation

In [149]:
n_points = 400
dt = 0.5
sigma0 = 5.0
k0 = 1.0
x0 = -150.0
x_begin = -200.0
x_end = 200.0
barrier_height = 1.0
barrier_width = 10.0

prob = np.zeros(n_points)
x, dx = np.linspace(x_begin, x_end, n_points, retstep=True)
norm = (2.0 * np.pi * sigma0 ** 2) ** (-0.25)
psi = np.exp(-(x - x0) ** 2 / (4.0 * sigma0 ** 2)) * np.exp(1.0j * k0 * x) * (2.0 * np.pi * sigma0 ** 2) ** (-0.25)

In [150]:
potential = np.zeros(x.shape)
potential[np.where((0.0 < x) & (x < barrier_width))] = barrier_height

In [151]:
h_diag = np.ones(n_points) / dx ** 2 + potential
h_non_diag = np.ones(n_points - 1) * (-0.5 / dx ** 2)
hamiltonian = sparse.diags([h_diag, h_non_diag, h_non_diag], [0, 1, -1])

In [152]:
implicit = (sparse.eye(n_points) - dt / 2.0j * hamiltonian).tocsc()
explicit = (sparse.eye(n_points) + dt / 2.0j * hamiltonian).tocsc()
evolution_matrix = ln.inv(implicit).dot(explicit).tocsr()

In [153]:
def evolve() -> np.array:
        global psi
        psi = evolution_matrix.dot(psi)
        prob = np.abs(psi) ** 2

        norm = prob.sum(axis=0)
        prob /= norm
        psi /= norm ** 0.5

        return prob

In [154]:
time = 0.0
fig, ax = plt.subplots()
plt.plot(x, potential * 0.1, color='r')

time_text = ax.text(0.05, 0.95, '', horizontalalignment='left', verticalalignment='top', transform=ax.transAxes)
line, = ax.plot(x, evolve())
ax.set_ylim(0, 0.2)
ax.set_xlabel('Position (a$_0$)')
ax.set_ylabel('Probability density (a$_0$)')

Text(0, 0.5, 'Probability density (a$_0$)')

In [155]:
def time_step() -> np.array:
    global time
    while True:
        time += dt
        time_text.set_text('Elapsed time: {:6.2f} fs'.format(time * 2.149e-2))
        yield evolve()

In [156]:
def update(data: np.array) -> tuple:
    line.set_ydata(data)
    return line, 

In [157]:
ani = animation.FuncAnimation(fig, update, time_step, interval=5, blit=False)

In [158]:
plt.show()